<a href="https://colab.research.google.com/github/navijose24/NeuroChain-Agent/blob/main/NeuroChain_multiagentic_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Build a multi-agent system called 'NeuroChain Agents' in Google Colab, following the detailed plan that includes initial setup, implementing core utilities, memory, tools (Google Search, Python Execution sandbox, File I/O), agents (Research, Search, Code-Runner), an Orchestrator agent, observability and evaluation hooks, context engineering, demo workflows, a Gradio UI, and project documentation.

## Initial Setup and Manual Steps

### Subtask:
Outline and present all required manual setup steps including library installations, environment variable configuration (e.g., GEMINI_API_KEY), and necessary Colab permissions (e.g., file access). Also, set up basic imports and the Gemini API client.


**Reasoning**:
Install all necessary Python libraries to begin the setup process.



In [1]:
pip install -q google-generativeai gradio python-dotenv nest_asyncio jupyter_client

### Mount Google Drive

To allow the system to save and load files from your Google Drive, you need to mount it. Run the following code block and follow the instructions to authorize access.

**Reasoning**:
Following the instruction to mount Google Drive, the next step is to execute the Python code that handles the mounting process, which is part of setting up Colab permissions for file access.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Configure Gemini API Key

To securely configure your `GEMINI_API_KEY`, follow these steps:

1.  **Obtain your API Key**: Go to [Google AI Studio](https://aistudio.google.com/app/apikey) and create your API key.
2.  **Add to Colab Secrets**: In Google Colab, click on the '🔑' icon (Secrets) in the left sidebar. Add your API key there with the name `GEMINI_API_KEY`.
3.  **Access in Notebook**: You can then access this key in your code using `import os; os.environ['GEMINI_API_KEY']` or `from google.colab import userdata; GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')`.

After setting it up in secrets, the next code block will load it.

In [17]:
import os
import google.generativeai as genai
from google.colab import userdata

try:
    # Attempt to load the GEMINI_API_KEY from Colab secrets
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

    if GEMINI_API_KEY:
        genai.configure(api_key=GEMINI_API_KEY)
        print("Gemini API client successfully configured with key from Colab secrets.")
    else:
        print("GEMINI_API_KEY not found in Colab secrets. Please ensure it's set correctly.")
        print("Follow the instructions in the markdown cell above to set your API key.")

except Exception as e:
    print(f"An error occurred while configuring the Gemini API client: {e}")
    print("Please ensure your GEMINI_API_KEY is correctly set in Colab secrets.")

Gemini API client successfully configured with key from Colab secrets.


## Implement Core Utilities and Memory

### Subtask:
Develop helper utilities, a basic logging system, and a memory management module to ensure context continuity across agents, including short-term and long-term memory capabilities.


**Reasoning**:
The first step is to configure a basic logging system as per the instructions, setting up output to both a file and the console with an INFO logging level.



In [18]:
import logging
import os

# Define the log file path
LOG_FILE = 'agent_activity.log'

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),  # Log to file
        logging.StreamHandler()       # Log to console
    ]
)

logger = logging.getLogger(__name__)
logger.info("Logging system initialized.")
print(f"Logging configured. Logs will be written to '{LOG_FILE}' and console.")

Logging configured. Logs will be written to 'agent_activity.log' and console.


**Reasoning**:
Following the instructions, the next step is to define the `MemoryManager` class, initializing it with attributes for short-term (list) and long-term (dictionary) memory storage.



In [19]:
class MemoryManager:
    def __init__(self):
        self.short_term_memory = []
        self.long_term_memory = {}
        logger.info("MemoryManager initialized.")
        print("MemoryManager class defined and initialized with short-term and long-term memory attributes.")


**Reasoning**:
The next step is to implement the specified methods within the `MemoryManager` class to handle adding, retrieving, and clearing short-term memory, and adding and retrieving long-term memory, as per the instructions.



In [20]:
class MemoryManager:
    def __init__(self):
        self.short_term_memory = []
        self.long_term_memory = {}
        logger.info("MemoryManager initialized.")
        print("MemoryManager class defined and initialized with short-term and long-term memory attributes.")

    def add_short_term_memory(self, item):
        self.short_term_memory.append(item)
        logger.debug(f"Added to short-term memory: {item}")

    def get_short_term_memory(self):
        logger.debug("Retrieving short-term memory.")
        return self.short_term_memory

    def clear_short_term_memory(self):
        self.short_term_memory = []
        logger.info("Short-term memory cleared.")

    def add_long_term_memory(self, key, value):
        self.long_term_memory[key] = value
        logger.info(f"Added to long-term memory: {key}")

    def get_long_term_memory(self, key):
        logger.debug(f"Retrieving long-term memory for key: {key}")
        return self.long_term_memory.get(key)

print("MemoryManager class updated with methods for short-term and long-term memory.")

MemoryManager class updated with methods for short-term and long-term memory.


**Reasoning**:
The final step in the instructions is to initialize an instance of the `MemoryManager` class to demonstrate its functionality.



In [21]:
memory_manager = MemoryManager()
print("MemoryManager instance initialized.")

MemoryManager class defined and initialized with short-term and long-term memory attributes.
MemoryManager instance initialized.


## Implement Google Search Tool

### Subtask:
Develop the Google Search tool, allowing agents to perform web searches.


**Reasoning**:
As per the instructions, the next step is to define the `google_search` function, which will take a query, log it, and return a placeholder string for its initial implementation.



In [22]:
def google_search(query: str) -> str:
    """
    Performs a simulated Google search for the given query.
    In a real-world scenario, this would integrate with a search API.
    """
    logger.info(f"Performing simulated Google search for query: '{query}'")
    # Placeholder for actual Google Search API integration
    return f"Simulated search results for: {query}"

print("google_search tool function defined.")

google_search tool function defined.


## Implement Python Execution Sandbox Tool

### Subtask:
Develop the Python Execution sandbox tool for safe code execution.


**Reasoning**:
As instructed, the next step is to import the `BlockingKernelClient` and `KernelManager` classes and then define the `execute_python_code` function, incorporating kernel management, code execution, output capture, and logging, and ensure clean shutdown.



In [23]:
import jupyter_client
import io
from contextlib import redirect_stdout, redirect_stderr

# Assuming `logger` is already defined from the logging setup

def execute_python_code(code: str) -> dict:
    """
    Executes Python code in a sandboxed environment using a Jupyter kernel.
    Returns a dictionary containing stdout, stderr, and an error flag.
    """
    logger.info(f"Attempting to execute Python code:\n{code}")

    # Use KernelManager to start a new kernel
    km = jupyter_client.KernelManager()
    km.start_kernel()

    try:
        # Connect a client to the kernel
        kc = km.client()
        kc.start_channels()
        kc.wait_for_ready()

        stdout_buffer = io.StringIO()
        stderr_buffer = io.StringIO()

        error_occurred = False
        output_messages = []

        try:
            # Execute the code interactively
            # We need to capture stdout/stderr from the kernel's execution
            # This is more complex than simple redirect_stdout/stderr
            # as it's an IPC communication.
            # For a true sandbox, we'd process messages from the kernel.
            # For this simplified version, we'll execute and then try to get output.
            msg_id = kc.execute(code)

            # Poll for messages until idle or timeout
            while True:
                try:
                    msg = kc.get_iopub_msg(timeout=1)
                    msg_type = msg['msg_type']
                    content = msg['content']

                    if msg_type == 'stream':
                        if content['name'] == 'stdout':
                            stdout_buffer.write(content['text'])
                        elif content['name'] == 'stderr':
                            stderr_buffer.write(content['text'])
                    elif msg_type == 'error':
                        stderr_buffer.write(f"{content['ename']}: {content['evalue']}\n")
                        for tb_line in content.get('traceback', []):
                            stderr_buffer.write(f"  {tb_line}")
                        error_occurred = True
                    elif msg_type == 'status' and content['execution_state'] == 'idle':
                        break # Kernel is idle, execution is complete
                except jupyter_client.timeout.Empty: # No more messages in this loop iteration
                    break
                except Exception as e:
                    stderr_buffer.write(f"Error processing kernel message: {e}\n")
                    error_occurred = True
                    break

        except Exception as e:
            stderr_buffer.write(f"Execution error: {e}\n")
            error_occurred = True

        stdout_output = stdout_buffer.getvalue().strip()
        stderr_output = stderr_buffer.getvalue().strip()

        if error_occurred or stderr_output: # Consider stderr as an error if it contains content
            logger.error(f"Python code execution failed. Stderr:\n{stderr_output}")
        else:
            logger.info(f"Python code executed successfully. Stdout:\n{stdout_output}")

        return {
            "stdout": stdout_output,
            "stderr": stderr_output,
            "error": error_occurred or bool(stderr_output)
        }
    finally:
        # Shut down the kernel and client cleanly
        try:
            if 'kc' in locals() and kc.is_alive():
                kc.stop_channels()
                kc.shutdown()
        except Exception as e:
            logger.error(f"Error stopping client channels/shutdown: {e}")
        try:
            if km.is_alive():
                km.shutdown_kernel()
        except Exception as e:
            logger.error(f"Error shutting down kernel manager: {e}")


print("Python execution sandbox function 'execute_python_code' defined.")

Python execution sandbox function 'execute_python_code' defined.


## Implement File I/O Capabilities

### Subtask:
Develop File I/O capabilities, allowing agents to read from and write to files.


**Reasoning**:
As instructed, the next step is to define the `read_file` function, which will handle reading content from a specified file, including logging and comprehensive error handling.



In [24]:
def read_file(file_path: str) -> str:
    """
    Reads content from a specified file.
    Handles FileNotFoundError and other IOError exceptions.
    """
    logger.info(f"Attempting to read file: {file_path}")
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        logger.info(f"Successfully read file: {file_path}")
        return content
    except FileNotFoundError:
        logger.error(f"FileNotFoundError: The file '{file_path}' was not found.")
        return ""
    except IOError as e:
        logger.error(f"IOError: Could not read file '{file_path}'. Error: {e}")
        return ""
    except Exception as e:
        logger.error(f"An unexpected error occurred while reading file '{file_path}'. Error: {e}")
        return ""

print("read_file function defined.")

read_file function defined.


**Reasoning**:
As instructed, the next step is to define the `write_file` function, which will handle writing content to a specified file, including logging and comprehensive error handling.



In [25]:
def write_file(file_path: str, content: str) -> bool:
    """
    Writes content to a specified file.
    Handles potential IOError exceptions.
    """
    logger.info(f"Attempting to write to file: {file_path}")
    try:
        with open(file_path, 'w') as f:
            f.write(content)
        logger.info(f"Successfully wrote to file: {file_path}")
        return True
    except IOError as e:
        logger.error(f"IOError: Could not write to file '{file_path}'. Error: {e}")
        return False
    except Exception as e:
        logger.error(f"An unexpected error occurred while writing to file '{file_path}'. Error: {e}")
        return False

print("write_file function defined.")

write_file function defined.


## Implement Agents (Research, Search, Code-Runner)

### Subtask:
Develop the Research Agent, defining its clear roles and functionalities, including its ability to break down queries, delegate tasks, and synthesize reports.


**Reasoning**:
Following the instructions, I will define the `ResearchAgent` class with an `__init__` method to initialize the memory manager and tools, and a placeholder `run` method that logs the research initiation and returns a status string.



In [26]:
import logging

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

class ResearchAgent:
    def __init__(self, memory_manager, tools):
        self.memory_manager = memory_manager
        self.tools = tools # Dictionary of available tools
        logger.info("ResearchAgent initialized.")

    def run(self, query: str) -> str:
        logger.info(f"Research task initiated for query: '{query}'")
        # Placeholder for actual research logic (breaking down queries, delegating, synthesizing)
        return f"Research task initiated for: {query}"

print("ResearchAgent class defined.")

ResearchAgent class defined.


**Reasoning**:
The previous step defined the `ResearchAgent` with a placeholder `run` method. To fulfill the subtask's requirement of defining its clear roles and functionalities, including its ability to break down queries, delegate tasks, and synthesize reports, I need to elaborate on the `run` method to incorporate actual research logic, utilizing the `google_search` tool and the `MemoryManager`.



In [27]:
class ResearchAgent:
    def __init__(self, memory_manager, tools):
        self.memory_manager = memory_manager
        self.tools = tools # Dictionary of available tools
        logger.info("ResearchAgent initialized.")

    def run(self, query: str) -> str:
        logger.info(f"Research task initiated for query: '{query}'")
        self.memory_manager.add_short_term_memory(f"Research query: {query}")

        # Step 1: Break down the query (simplified for now)
        search_query = f"Information about {query}"
        logger.info(f"Breaking down query, performing search for: '{search_query}'")
        self.memory_manager.add_short_term_memory(f"Sub-task: Perform Google search for '{search_query}'")

        # Step 2: Delegate search task
        search_results = self.tools['google_search'](search_query)
        logger.info("Search task delegated and results obtained.")
        self.memory_manager.add_short_term_memory(f"Search results: {search_results}")

        # Step 3: Synthesize reports (simplified for now)
        report = f"Research Report on '{query}':\n\n" \
                 f"Initial Query: {query}\n" \
                 f"Search Performed: {search_query}\n" \
                 f"Key Findings (from simulated search): {search_results}\n\n"

        self.memory_manager.add_long_term_memory(f"research_report_{query.replace(' ', '_')}", report)
        logger.info("Research synthesis complete and stored in long-term memory.")

        return report

print("ResearchAgent class updated with basic research logic in the run method.")

ResearchAgent class updated with basic research logic in the run method.


## Implement Agents (Research, Search, Code-Runner)

### Subtask:
Develop the Search Agent, detailing its roles in fetching and filtering information.


**Reasoning**:
As instructed, I will define the `SearchAgent` class, including its `__init__` method to initialize the memory manager and tools, and a `run` method to execute a Google search, log activity, and store results in short-term memory.



In [28]:
class SearchAgent:
    def __init__(self, memory_manager, tools):
        self.memory_manager = memory_manager
        self.tools = tools # Dictionary of available tools
        logger.info("SearchAgent initialized.")

    def run(self, query: str) -> str:
        logger.info(f"Search task initiated for query: '{query}'")
        self.memory_manager.add_short_term_memory(f"Search query: {query}")

        search_results = self.tools['google_search'](query)
        logger.info("Search task executed.")
        self.memory_manager.add_short_term_memory(f"Search results: {search_results}")

        return search_results

print("SearchAgent class defined.")

SearchAgent class defined.


**Reasoning**:
Following the instructions, I will define the `CodeRunnerAgent` class, including its `__init__` method to initialize the memory manager and tools, and a `run` method to execute Python code, log activity, and store results in short-term memory.



In [29]:
class CodeRunnerAgent:
    def __init__(self, memory_manager, tools):
        self.memory_manager = memory_manager
        self.tools = tools # Dictionary of available tools
        logger.info("CodeRunnerAgent initialized.")

    def run(self, code: str) -> dict:
        logger.info(f"Code execution task initiated for code:\n{code}")
        self.memory_manager.add_short_term_memory(f"Code to execute: {code}")

        execution_results = self.tools['execute_python_code'](code)
        logger.info("Code execution task completed.")
        self.memory_manager.add_short_term_memory(f"Code execution results: {execution_results}")

        return execution_results

print("CodeRunnerAgent class defined.")

CodeRunnerAgent class defined.


## Implement Orchestrator Agent

### Subtask:
Develop the Orchestrator Agent, defining its control mechanisms for workflows, tracking agent states, routing messages, and logging.


**Reasoning**:
As per the instructions, I will define the `OrchestratorAgent` class, including its `__init__` method to initialize the memory manager, agents, and tools, and a basic `run` method to log activity and return a placeholder string.



In [30]:
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        # Placeholder for actual orchestration logic
        return f"Orchestration task initiated for: {task}"

print("OrchestratorAgent class defined.")

OrchestratorAgent class defined.


## Implement Orchestrator Agent

### Subtask:
Enhance the Orchestrator Agent's `run` method to include basic decision-making and task delegation to specialized agents based on the input task.


**Reasoning**:
As instructed, I will modify the `OrchestratorAgent` class, specifically enhancing its `run` method to include conditional logic for task delegation to the appropriate specialized agents (`ResearchAgent`, `SearchAgent`, `CodeRunnerAgent`) based on keywords in the input task.



In [31]:
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")

        task_lower = task.lower()

        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            return self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            return self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            # Assuming the code to execute is directly in the task string after the trigger phrase
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                return self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                return "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            return f"Orchestration completed for: {task}. No specialized agent invoked."

print("OrchestratorAgent class updated with enhanced run method for task delegation.")

OrchestratorAgent class updated with enhanced run method for task delegation.


## Develop Tracing Mechanism

### Subtask:
Develop a basic tracing mechanism to track the flow of execution across agents.


**Reasoning**:
As instructed, I will define the `Tracer` class with its `__init__` method and the `start_trace`, `add_event`, `get_trace`, and `clear_traces` methods to establish the basic tracing mechanism.



In [32]:
import datetime

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

class Tracer:
    def __init__(self):
        self.trace_events = []
        logger.info("Tracer initialized.")

    def start_trace(self, task_id: str, description: str):
        event = {
            "timestamp": datetime.datetime.now().isoformat(),
            "task_id": task_id,
            "event_type": "start_trace",
            "description": description,
            "details": {}
        }
        self.trace_events.append(event)
        logger.info(f"Started trace for task_id: {task_id} - {description}")

    def add_event(self, task_id: str, event_type: str, details: dict):
        event = {
            "timestamp": datetime.datetime.now().isoformat(),
            "task_id": task_id,
            "event_type": event_type,
            "details": details
        }
        self.trace_events.append(event)
        logger.debug(f"Added event to trace for task_id: {task_id}, type: {event_type}, details: {details}")

    def get_trace(self, task_id: str) -> list:
        return [event for event in self.trace_events if event["task_id"] == task_id]

    def clear_traces(self):
        self.trace_events = []
        logger.info("All traces cleared.")

print("Tracer class defined with start_trace, add_event, get_trace, and clear_traces methods.")

Tracer class defined with start_trace, add_event, get_trace, and clear_traces methods.


## Integrate Observability and Evaluation Hooks

### Subtask:
Integrate the tracing mechanism into the agents and orchestrator.


**Reasoning**:
As instructed, the first step is to initialize an instance of the `Tracer` class to make it available for integration with the agents.



In [33]:
tracer = Tracer()
print("Tracer instance initialized.")

Tracer instance initialized.


**Reasoning**:
As instructed, I will update the `__init__` methods of `ResearchAgent`, `SearchAgent`, `CodeRunnerAgent`, and `OrchestratorAgent` to accept a `tracer` instance as an argument and store it, ensuring the tracing mechanism can be passed to all agents.



In [34]:
class ResearchAgent:
    def __init__(self, memory_manager, tools, tracer):
        self.memory_manager = memory_manager
        self.tools = tools
        self.tracer = tracer # Store the tracer instance
        logger.info("ResearchAgent initialized.")

    def run(self, query: str) -> str:
        logger.info(f"Research task initiated for query: '{query}'")
        self.memory_manager.add_short_term_memory(f"Research query: {query}")
        self.tracer.add_event(
            task_id=query, # Using query as a basic task_id for now
            event_type="research_query_received",
            details={
                "query": query
            }
        )

        # Step 1: Break down the query (simplified for now)
        search_query = f"Information about {query}"
        logger.info(f"Breaking down query, performing search for: '{search_query}'")
        self.memory_manager.add_short_term_memory(f"Sub-task: Perform Google search for '{search_query}'")
        self.tracer.add_event(
            task_id=query,
            event_type="subtask_delegated_search",
            details={
                "sub_query": search_query,
                "delegated_to": "SearchAgent (implicit)"
            }
        )

        # Step 2: Delegate search task
        search_results = self.tools['google_search'](search_query)
        logger.info("Search task delegated and results obtained.")
        self.memory_manager.add_short_term_memory(f"Search results: {search_results}")
        self.tracer.add_event(
            task_id=query,
            event_type="search_results_obtained",
            details={
                "results_summary": search_results[:100] + "..." if len(search_results) > 100 else search_results
            }
        )

        # Step 3: Synthesize reports (simplified for now)
        report = f"Research Report on '{query}':\n\n" \
                 f"Initial Query: {query}\n" \
                 f"Search Performed: {search_query}\n" \
                 f"Key Findings (from simulated search): {search_results}\n\n"

        self.memory_manager.add_long_term_memory(f"research_report_{query.replace(' ', '_')}", report)
        logger.info("Research synthesis complete and stored in long-term memory.")
        self.tracer.add_event(
            task_id=query,
            event_type="report_synthesized",
            details={
                "report_title": f"Research Report on '{query}'",
                "memory_key": f"research_report_{query.replace(' ', '_')}"
            }
        )

        return report


class SearchAgent:
    def __init__(self, memory_manager, tools, tracer):
        self.memory_manager = memory_manager
        self.tools = tools
        self.tracer = tracer # Store the tracer instance
        logger.info("SearchAgent initialized.")

    def run(self, query: str) -> str:
        logger.info(f"Search task initiated for query: '{query}'")
        self.memory_manager.add_short_term_memory(f"Search query: {query}")
        self.tracer.add_event(
            task_id=query, # Using query as a basic task_id
            event_type="search_query_received",
            details={
                "query": query
            }
        )

        search_results = self.tools['google_search'](query)
        logger.info("Search task executed.")
        self.memory_manager.add_short_term_memory(f"Search results: {search_results}")
        self.tracer.add_event(
            task_id=query,
            event_type="search_executed",
            details={
                "query": query,
                "results_summary": search_results[:100] + "..." if len(search_results) > 100 else search_results
            }
        )

        return search_results


class CodeRunnerAgent:
    def __init__(self, memory_manager, tools, tracer):
        self.memory_manager = memory_manager
        self.tools = tools
        self.tracer = tracer # Store the tracer instance
        logger.info("CodeRunnerAgent initialized.")

    def run(self, code: str) -> dict:
        logger.info(f"Code execution task initiated for code:\n{code}")
        self.memory_manager.add_short_term_memory(f"Code to execute: {code}")
        self.tracer.add_event(
            task_id=code[:20].replace('\n', ' ') + "...", # Using a snippet of code as task_id
            event_type="code_execution_request_received",
            details={
                "code_snippet": code[:100] + "..." if len(code) > 100 else code
            }
        )

        execution_results = self.tools['execute_python_code'](code)
        logger.info("Code execution task completed.")
        self.memory_manager.add_short_term_memory(f"Code execution results: {execution_results}")
        self.tracer.add_event(
            task_id=code[:20].replace('\n', ' ') + "...",
            event_type="code_execution_completed",
            details={
                "stdout": execution_results.get('stdout', '')[:100] + "..." if len(execution_results.get('stdout', '')) > 100 else execution_results.get('stdout', ''),
                "stderr": execution_results.get('stderr', '')[:100] + "..." if len(execution_results.get('stderr', '')) > 100 else execution_results.get('stderr', ''),
                "error": execution_results.get('error', False)
            }
        )

        return execution_results


class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents
        self.tools = tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            return self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            return self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                return self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                return "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            return f"Orchestration completed for: {task}. No specialized agent invoked."

print("Agent classes (ResearchAgent, SearchAgent, CodeRunnerAgent, OrchestratorAgent) updated to accept tracer and log events.")

Agent classes (ResearchAgent, SearchAgent, CodeRunnerAgent, OrchestratorAgent) updated to accept tracer and log events.


## Develop Context Engineering and Compaction

### Subtask:
Implement mechanisms for context engineering and compaction to efficiently manage and process information across the multi-agent system.


**Reasoning**:
As instructed, I will define the `ContextManager` class with its `__init__` method, which will take a `MemoryManager` instance, initialize `self.conversation_history` and `self.max_context_length`, and set up logging.



In [35]:
import datetime
import logging

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

class ContextManager:
    def __init__(self, memory_manager):
        self.memory_manager = memory_manager
        self.conversation_history = []
        self.max_context_length = 4000  # Example token limit
        logger.info("ContextManager initialized.")
        print("ContextManager class defined and initialized.")


**Reasoning**:
Following the instructions, I will implement the `add_message` method within the `ContextManager` class to record conversation history with details like sender, message, type, and timestamp.



In [36]:
class ContextManager:
    def __init__(self, memory_manager):
        self.memory_manager = memory_manager
        self.conversation_history = []
        self.max_context_length = 4000  # Example token limit
        logger.info("ContextManager initialized.")
        print("ContextManager class defined and initialized.")

    def add_message(self, sender: str, message: str, message_type: str = "info"):
        timestamp = datetime.datetime.now().isoformat()
        event = {
            "timestamp": timestamp,
            "sender": sender,
            "message": message,
            "type": message_type
        }
        self.conversation_history.append(event)
        logger.info(f"ContextManager: Added message from {sender} (Type: {message_type}).")

print("ContextManager class updated with add_message method.")

ContextManager class updated with add_message method.


**Reasoning**:
As instructed, I will implement the `get_current_context` method within the `ContextManager` class, which will return the current `conversation_history` list.



In [37]:
class ContextManager:
    def __init__(self, memory_manager):
        self.memory_manager = memory_manager
        self.conversation_history = []
        self.max_context_length = 4000  # Example token limit
        logger.info("ContextManager initialized.")
        print("ContextManager class defined and initialized.")

    def add_message(self, sender: str, message: str, message_type: str = "info"):
        timestamp = datetime.datetime.now().isoformat()
        event = {
            "timestamp": timestamp,
            "sender": sender,
            "message": message,
            "type": message_type
        }
        self.conversation_history.append(event)
        logger.info(f"ContextManager: Added message from {sender} (Type: {message_type}).")

    def get_current_context(self) -> list:
        logger.debug("Retrieving current conversation context.")
        return self.conversation_history

print("ContextManager class updated with get_current_context method.")

ContextManager class updated with get_current_context method.


**Reasoning**:
As instructed, I will implement the `compact_context` method within the `ContextManager` class. This method will calculate the total length of messages, and if it exceeds `self.max_context_length`, it will remove the oldest messages until the context fits, logging the compaction action.



In [38]:
class ContextManager:
    def __init__(self, memory_manager):
        self.memory_manager = memory_manager
        self.conversation_history = []
        self.max_context_length = 4000  # Example token limit
        logger.info("ContextManager initialized.")
        print("ContextManager class defined and initialized.")

    def add_message(self, sender: str, message: str, message_type: str = "info"):
        timestamp = datetime.datetime.now().isoformat()
        event = {
            "timestamp": timestamp,
            "sender": sender,
            "message": message,
            "type": message_type
        }
        self.conversation_history.append(event)
        logger.info(f"ContextManager: Added message from {sender} (Type: {message_type}).")

    def get_current_context(self) -> list:
        logger.debug("Retrieving current conversation context.")
        return self.conversation_history

    def compact_context(self):
        current_length = sum(len(entry['message']) for entry in self.conversation_history)
        if current_length > self.max_context_length:
            logger.info(f"Context length ({current_length}) exceeds max_context_length ({self.max_context_length}). Compacting context.")
            while current_length > self.max_context_length and len(self.conversation_history) > 1:
                removed_message = self.conversation_history.pop(0) # Remove oldest message
                current_length = sum(len(entry['message']) for entry in self.conversation_history)
                logger.debug(f"Removed oldest message for compaction: {removed_message['message'][:50]}...")
            logger.info(f"Context compacted. New length: {current_length}")
        else:
            logger.debug("Context length is within limits. No compaction needed.")

print("ContextManager class updated with compact_context method.")

ContextManager class updated with compact_context method.


**Reasoning**:
As instructed, I will initialize an instance of the `ContextManager` class, passing the existing `memory_manager` instance to it, to complete the setup of context engineering.



In [39]:
context_manager = ContextManager(memory_manager)
print("ContextManager instance initialized.")

ContextManager class defined and initialized.
ContextManager instance initialized.


## Create Demo Workflows

### Subtask:
Develop the specified demo workflows: a research task, a web search task, and a code-execution task, showcasing the integrated functionality of the NeuroChain Agents system.


**Reasoning**:
As instructed, I will initialize the necessary components, populate the tools dictionary, instantiate all agents (Research, Search, CodeRunner, Orchestrator), and then demonstrate the execution of research, web search, and code-execution demo tasks using the Orchestrator Agent, printing their results and traces to showcase integrated functionality.



In [40]:
print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components
tools = {}
agents = {}

# Instantiate MemoryManager (already done, but re-init for clarity in this block)
memory_manager = MemoryManager()
# Instantiate Tracer (already done, but re-init for clarity in this block)
tracer = Tracer()
# Instantiate ContextManager (already done, but re-init for clarity in this block)
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary
tools['google_search'] = google_search
tools['execute_python_code'] = execute_python_code
tools['read_file'] = read_file
tools['write_file'] = write_file

print("Tools dictionary populated.")

# 3. Instantiate agents
research_agent = ResearchAgent(memory_manager, tools, tracer)
search_agent = SearchAgent(memory_manager, tools, tracer)
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents['research_agent'] = research_agent
agents['search_agent'] = search_agent
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.")

# 4. Instantiate the OrchestratorAgent
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_result = orchestrator_agent.run(research_task)
print(f"Research Task Result:\n{research_result}")
research_trace = tracer.get_trace(f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}".split('_')[1] + "_" + research_task.replace(' ', '_').replace('.', '')[:10]) # Simplified task_id for demo
# Note: The task_id generated by the orchestrator.run is dynamic. For reliable retrieval, we should capture the exact task_id. For now, assuming a general lookup.

# A more robust way to get the trace for the last orchestrated task:
# The OrchestratorAgent.run generates a task_id. We need to store and return it.
# For this demo, let's assume the last trace added by the orchestrator is relevant.
# In a real scenario, the run method should return the generated task_id.
# For now, let's fetch all traces and identify the latest orchestration one.
all_traces = tracer.trace_events
last_orchestration_task_id = None
for event in reversed(all_traces):
    if event.get('event_type') == 'start_trace' and event.get('description').startswith('Orchestrating task: research'):
        last_orchestration_task_id = event['task_id']
        break

if last_orchestration_task_id:
    print(f"\nTrace for Research Task (ID: {last_orchestration_task_id}):")
    for event in tracer.get_trace(last_orchestration_task_id):
        print(event)
else:
    print("Could not retrieve trace for research task.")

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_result = orchestrator_agent.run(search_task)
print(f"Web Search Task Result:\n{search_result}")

last_orchestration_task_id = None
for event in reversed(all_traces):
    if event.get('event_type') == 'start_trace' and event.get('description').startswith('Orchestrating task: search for'):
        last_orchestration_task_id = event['task_id']
        break

if last_orchestration_task_id:
    print(f"\nTrace for Web Search Task (ID: {last_orchestration_task_id}):")
    for event in tracer.get_trace(last_orchestration_task_id):
        print(event)
else:
    print("Could not retrieve trace for web search task.")

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_result = orchestrator_agent.run(code_task)
print(f"Code-Execution Task Result:\n{code_result}")

last_orchestration_task_id = None
for event in reversed(all_traces):
    if event.get('event_type') == 'start_trace' and event.get('description').startswith('Orchestrating task: execute code'):
        last_orchestration_task_id = event['task_id']
        break

if last_orchestration_task_id:
    print(f"\nTrace for Code-Execution Task (ID: {last_orchestration_task_id}):")
    for event in tracer.get_trace(last_orchestration_task_id):
        print(event)
else:
    print("Could not retrieve trace for code-execution task.")

print("\nDemo workflows completed.")

Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary populated.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Research Task (ID: orchestration_20251203194954824368):
{'timestamp': '2025-12-03T19:49:54.824413', 'task_id': 'orchestration_20251

ERROR:__main__:Error stopping client channels/shutdown: 


Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203194954826284):
{'timestamp': '2025-12-03T19:49:54.826306', 'task_id': 'orchestration_20251203194954826284', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:49:54.826318', 'task_id': 'orchestration_20251203194954826284', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:49:54.826337', 'task_id': 'orchestration_20251203194954826284', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The previous execution indicated an error during the shutdown of the Jupyter kernel client. This error (`ERROR:__main__:Error stopping client channels/shutdown:`) suggests issues with the `kc.stop_channels()` or `kc.shutdown()` calls. I will update the `execute_python_code` function to ensure more robust and explicit error handling during client and kernel shutdown, logging more informative messages in case of failure. Additionally, the demo workflow needs to reliably retrieve traces by ensuring the `OrchestratorAgent.run` method returns the generated `task_id`, and then using this `task_id` for trace retrieval.



In [41]:
import jupyter_client
import io
from contextlib import redirect_stdout, redirect_stderr

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

def execute_python_code(code: str) -> dict:
    """
    Executes Python code in a sandboxed environment using a Jupyter kernel.
    Returns a dictionary containing stdout, stderr, and an error flag.
    """
    logger.info(f"Attempting to execute Python code:\n{code}")

    km = None
    kc = None
    try:
        # Use KernelManager to start a new kernel
        km = jupyter_client.KernelManager()
        km.start_kernel()

        # Connect a client to the kernel
        kc = km.client()
        kc.start_channels()
        kc.wait_for_ready()

        stdout_buffer = io.StringIO()
        stderr_buffer = io.StringIO()

        error_occurred = False

        try:
            # Execute the code interactively
            msg_id = kc.execute(code)

            # Poll for messages until idle or timeout
            while True:
                try:
                    msg = kc.get_iopub_msg(timeout=1)
                    msg_type = msg['msg_type']
                    content = msg['content']

                    if msg_type == 'stream':
                        if content['name'] == 'stdout':
                            stdout_buffer.write(content['text'])
                        elif content['name'] == 'stderr':
                            stderr_buffer.write(content['text'])
                    elif msg_type == 'error':
                        stderr_buffer.write(f"{content['ename']}: {content['evalue']}\n")
                        for tb_line in content.get('traceback', []):
                            stderr_buffer.write(f"  {tb_line}")
                        error_occurred = True
                    elif msg_type == 'status' and content['execution_state'] == 'idle':
                        break # Kernel is idle, execution is complete
                except jupyter_client.timeout.Empty: # No more messages in this loop iteration
                    break
                except Exception as e:
                    stderr_buffer.write(f"Error processing kernel message: {e}\n")
                    error_occurred = True
                    break

        except Exception as e:
            stderr_buffer.write(f"Execution error: {e}\n")
            error_occurred = True

        stdout_output = stdout_buffer.getvalue().strip()
        stderr_output = stderr_buffer.getvalue().strip()

        if error_occurred or stderr_output: # Consider stderr as an error if it contains content
            logger.error(f"Python code execution failed. Stderr:\n{stderr_output}")
        else:
            logger.info(f"Python code executed successfully. Stdout:\n{stdout_output}")

        return {
            "stdout": stdout_output,
            "stderr": stderr_output,
            "error": error_occurred or bool(stderr_output)
        }
    finally:
        # Shut down the client first, then the kernel manager
        if kc:
            try:
                # Stop client channels gracefully
                if kc.is_alive():
                    kc.stop_channels()
                    logger.debug("Client channels stopped.")
                # Attempt to shut down the client
                kc.shutdown()
                logger.debug("Client shut down.")
            except Exception as e:
                # Log the exception, ensuring a message is always present
                error_msg = str(e) if str(e) else "Unknown error during client shutdown"
                logger.error(f"Error during client shutdown: {error_msg}")

        if km:
            try:
                # Shut down the kernel manager
                if km.is_alive():
                    km.shutdown_kernel()
                    logger.debug("Kernel manager shut down.")
            except Exception as e:
                # Log the exception, ensuring a message is always present
                error_msg = str(e) if str(e) else "Unknown error during kernel manager shutdown"
                logger.error(f"Error during kernel manager shutdown: {error_msg}")


print("Python execution sandbox function 'execute_python_code' defined.")

Python execution sandbox function 'execute_python_code' defined.


**Reasoning**:
To enable reliable trace retrieval, I will modify the `OrchestratorAgent`'s `run` method to return the `task_id` it generates. Then, I will update the demo workflow code to capture this `task_id` and use it to accurately retrieve the trace for each executed task, showcasing the integrated functionality with proper observability.



In [42]:
import datetime

# Re-define OrchestratorAgent to return task_id
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# Re-initialize the OrchestratorAgent with the updated class definition
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)
print("OrchestratorAgent re-instantiated with updated definition.")

print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components (already done, ensuring fresh start for demo)
# Instantiate MemoryManager (already done, but re-init for clarity in this block)
memory_manager = MemoryManager()
# Instantiate Tracer (already done, but re-init for clarity in this block)
tracer = Tracer()
# Instantiate ContextManager (already done, but re-init for clarity in this block)
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (assuming already populated from previous execution)
# tools['google_search'] = google_search
# tools['execute_python_code'] = execute_python_code
# tools['read_file'] = read_file
# tools['write_file'] = write_file

print("Tools dictionary (re)populated.")

# 3. Instantiate agents (assuming already instantiated, but re-instantiate with tracer)
research_agent = ResearchAgent(memory_manager, tools, tracer)
search_agent = SearchAgent(memory_manager, tools, tracer)
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents['research_agent'] = research_agent
agents['search_agent'] = search_agent
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.")

# 4. Instantiate the OrchestratorAgent
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")

OrchestratorAgent class updated to return result and task_id.
OrchestratorAgent re-instantiated with updated definition.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary (re)populated.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Resear

ERROR:__main__:Error stopping client channels/shutdown: 


Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203195055994678):
{'timestamp': '2025-12-03T19:50:55.996388', 'task_id': 'orchestration_20251203195055994678', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:50:55.996413', 'task_id': 'orchestration_20251203195055994678', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:50:55.996440', 'task_id': 'orchestration_20251203195055994678', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The previous execution indicated an error during the shutdown of the Jupyter kernel client. This error (`ERROR:__main__:Error stopping client channels/shutdown:`) suggests issues with the `kc.stop_channels()` or `kc.shutdown()` calls. I will update the `execute_python_code` function to ensure more robust and explicit error handling during client and kernel shutdown, logging more informative messages in case of failure. Additionally, the demo workflow needs to reliably retrieve traces by ensuring the `OrchestratorAgent.run` method returns the generated `task_id`, and then using this `task_id` for trace retrieval.



In [43]:
import datetime

# Re-define OrchestratorAgent to return task_id
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# Re-initialize the OrchestratorAgent with the updated class definition
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)
print("OrchestratorAgent re-instantiated with updated definition.")

print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components (already done, ensuring fresh start for demo)
# Instantiate MemoryManager (already done, but re-init for clarity in this block)
memory_manager = MemoryManager()
# Instantiate Tracer (already done, but re-init for clarity in this block)
tracer = Tracer()
# Instantiate ContextManager (already done, but re-init for clarity in this block)
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (assuming already populated from previous execution)
# tools['google_search'] = google_search
# tools['execute_python_code'] = execute_python_code
# tools['read_file'] = read_file
# tools['write_file'] = write_file

print("Tools dictionary (re)populated.")

# 3. Instantiate agents (assuming already instantiated, but re-instantiate with tracer)
research_agent = ResearchAgent(memory_manager, tools, tracer)
search_agent = SearchAgent(memory_manager, tools, tracer)
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents['research_agent'] = research_agent
agents['search_agent'] = search_agent
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.")

# 4. Instantiate the OrchestratorAgent
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")

OrchestratorAgent class updated to return result and task_id.
OrchestratorAgent re-instantiated with updated definition.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary (re)populated.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Resear

ERROR:__main__:Error stopping client channels/shutdown: 


Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203195125446913):
{'timestamp': '2025-12-03T19:51:25.446927', 'task_id': 'orchestration_20251203195125446913', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:51:25.446934', 'task_id': 'orchestration_20251203195125446913', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:51:25.446946', 'task_id': 'orchestration_20251203195125446913', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The persistent `Error stopping client channels/shutdown:` error in the `execute_python_code` function suggests that the current shutdown logic for the Jupyter client and kernel is not robust enough. I will modify the `finally` block to directly attempt `kc.stop_channels()` and `kc.shutdown()` with more precise error logging using `repr(e)` to capture detailed exception information, and ensure the kernel manager is shut down last.



In [44]:
import jupyter_client
import io
from contextlib import redirect_stdout, redirect_stderr
import time

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

def execute_python_code(code: str) -> dict:
    """
    Executes Python code in a sandboxed environment using a Jupyter kernel.
    Returns a dictionary containing stdout, stderr, and an error flag.
    """
    logger.info(f"Attempting to execute Python code:\n{code}")

    km = None
    kc = None
    try:
        # Use KernelManager to start a new kernel
        km = jupyter_client.KernelManager()
        km.start_kernel()

        # Connect a client to the kernel
        kc = km.client()
        kc.start_channels()
        kc.wait_for_ready()

        stdout_buffer = io.StringIO()
        stderr_buffer = io.StringIO()

        error_occurred = False

        try:
            # Execute the code interactively
            msg_id = kc.execute(code)

            # Poll for messages until idle or timeout
            while True:
                try:
                    msg = kc.get_iopub_msg(timeout=1)
                    msg_type = msg['msg_type']
                    content = msg['content']

                    if msg_type == 'stream':
                        if content['name'] == 'stdout':
                            stdout_buffer.write(content['text'])
                        elif content['name'] == 'stderr':
                            stderr_buffer.write(content['text'])
                    elif msg_type == 'error':
                        stderr_buffer.write(f"{content['ename']}: {content['evalue']}\n")
                        for tb_line in content.get('traceback', []):
                            stderr_buffer.write(f"  {tb_line}")
                        error_occurred = True
                    elif msg_type == 'status' and content['execution_state'] == 'idle':
                        break # Kernel is idle, execution is complete
                except jupyter_client.timeout.Empty: # No more messages in this loop iteration
                    break
                except Exception as e:
                    stderr_buffer.write(f"Error processing kernel message: {e}\n")
                    error_occurred = True
                    break

        except Exception as e:
            stderr_buffer.write(f"Execution error: {e}\n")
            error_occurred = True

        stdout_output = stdout_buffer.getvalue().strip()
        stderr_output = stderr_buffer.getvalue().strip()

        if error_occurred or stderr_output: # Consider stderr as an error if it contains content
            logger.error(f"Python code execution failed. Stderr:\n{stderr_output}")
        else:
            logger.info(f"Python code executed successfully. Stdout:\n{stdout_output}")

        return {
            "stdout": stdout_output,
            "stderr": stderr_output,
            "error": error_occurred or bool(stderr_output)
        }
    finally:
        # Shut down the client first, then the kernel manager
        if kc:
            try:
                # Attempt to stop client channels
                kc.stop_channels()
                logger.debug("Client channels stopped.")
            except Exception as e:
                error_msg = f"Error stopping client channels: {e!r}"
                logger.warning(f"{error_msg}")

            try:
                # Attempt to shut down the client
                kc.shutdown()
                logger.debug("Client shut down.")
            except Exception as e:
                error_msg = f"Error during client shutdown: {e!r}"
                logger.warning(f"{error_msg}")

        if km:
            try:
                # Shut down the kernel manager
                km.shutdown_kernel()
                logger.debug("Kernel manager shut down.")
            except Exception as e:
                error_msg = f"Error during kernel manager shutdown: {e!r}"
                logger.error(f"{error_msg}")


print("Python execution sandbox function 'execute_python_code' defined with improved shutdown handling.")

Python execution sandbox function 'execute_python_code' defined with improved shutdown handling.


**Reasoning**:
The previous steps updated the `execute_python_code` function and modified the `OrchestratorAgent` to return `task_id`. The next step is to re-run the demo workflows to verify these changes and ensure the system functions as expected without shutdown errors, and with accurate trace retrieval.



In [45]:
import datetime

# Re-define OrchestratorAgent to return task_id
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# Re-initialize the OrchestratorAgent with the updated class definition
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)
print("OrchestratorAgent re-instantiated with updated definition.")

print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components (already done, ensuring fresh start for demo)
# Instantiate MemoryManager (already done, but re-init for clarity in this block)
memory_manager = MemoryManager()
# Instantiate Tracer (already done, but re-init for clarity in this block)
tracer = Tracer()
# Instantiate ContextManager (already done, but re-init for clarity in this block)
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (assuming already populated from previous execution)
# tools['google_search'] = google_search
# tools['execute_python_code'] = execute_python_code
# tools['read_file'] = read_file
# tools['write_file'] = write_file

print("Tools dictionary (re)populated.")

# 3. Instantiate agents (assuming already instantiated, but re-instantiate with tracer)
research_agent = ResearchAgent(memory_manager, tools, tracer)
sn_search_agent = SearchAgent(memory_manager, tools, tracer)
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents['research_agent'] = research_agent
agents['search_agent'] = search_agent
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.")

# 4. Instantiate the OrchestratorAgent
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")

OrchestratorAgent class updated to return result and task_id.
OrchestratorAgent re-instantiated with updated definition.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary (re)populated.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Resear

ERROR:__main__:Error stopping client channels/shutdown: 


Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203195218902494):
{'timestamp': '2025-12-03T19:52:18.902507', 'task_id': 'orchestration_20251203195218902494', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:52:18.902514', 'task_id': 'orchestration_20251203195218902494', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:52:18.902527', 'task_id': 'orchestration_20251203195218902494', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The previous steps updated the `execute_python_code` function and modified the `OrchestratorAgent` to return `task_id`. The next step is to re-run the demo workflows to verify these changes and ensure the system functions as expected without shutdown errors, and with accurate trace retrieval.



In [46]:
import datetime

# Re-define OrchestratorAgent to return task_id
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# Re-initialize the OrchestratorAgent with the updated class definition
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)
print("OrchestratorAgent re-instantiated with updated definition.")

print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components (already done, ensuring fresh start for demo)
# Instantiate MemoryManager (already done, but re-init for clarity in this block)
memory_manager = MemoryManager()
# Instantiate Tracer (already done, but re-init for clarity in this block)
tracer = Tracer()
# Instantiate ContextManager (already done, but re-init for clarity in this block)
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (assuming already populated from previous execution)
# tools['google_search'] = google_search
# tools['execute_python_code'] = execute_python_code
# tools['read_file'] = read_file
# tools['write_file'] = write_file

print("Tools dictionary (re)populated.")

# 3. Instantiate agents (assuming already instantiated, but re-instantiate with tracer)
research_agent = ResearchAgent(memory_manager, tools, tracer)
sn_search_agent = SearchAgent(memory_manager, tools, tracer)
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents['research_agent'] = research_agent
agents['search_agent'] = sn_search_agent
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.")

# 4. Instantiate the OrchestratorAgent
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")


OrchestratorAgent class updated to return result and task_id.
OrchestratorAgent re-instantiated with updated definition.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary (re)populated.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Resear

ERROR:__main__:Error stopping client channels/shutdown: 


Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203195242417278):
{'timestamp': '2025-12-03T19:52:42.417309', 'task_id': 'orchestration_20251203195242417278', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:52:42.417317', 'task_id': 'orchestration_20251203195242417278', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:52:42.417330', 'task_id': 'orchestration_20251203195242417278', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The previous demo workflow failed because the `tools` dictionary was not explicitly re-populated with the updated `execute_python_code` function, and there was a typo where `sn_search_agent` was instantiated but `search_agent` was used in the `agents` dictionary. I will fix these issues in the demo workflow code to ensure the latest `execute_python_code` is used and all agents are correctly assigned.



In [47]:
import datetime

# Re-define OrchestratorAgent to return task_id
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# Re-initialize the OrchestratorAgent with the updated class definition
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)
print("OrchestratorAgent re-instantiated with updated definition.")

print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components (already done, ensuring fresh start for demo)
memory_manager = MemoryManager()
tracer = Tracer()
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (explicitly re-populate)
tools['google_search'] = google_search
tools['execute_python_code'] = execute_python_code
tools['read_file'] = read_file
tools['write_file'] = write_file

print("Tools dictionary (re)populated.")

# 3. Instantiate agents (re-instantiate with tracer)
research_agent = ResearchAgent(memory_manager, tools, tracer)
search_agent = SearchAgent(memory_manager, tools, tracer) # Fixed variable name
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents['research_agent'] = research_agent
agents['search_agent'] = search_agent # Fixed assignment
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.")

# 4. Instantiate the OrchestratorAgent
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")


OrchestratorAgent class updated to return result and task_id.
OrchestratorAgent re-instantiated with updated definition.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary (re)populated.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Resear

Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203195311169717):
{'timestamp': '2025-12-03T19:53:11.169745', 'task_id': 'orchestration_20251203195311169717', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:53:11.169759', 'task_id': 'orchestration_20251203195311169717', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:53:11.169780', 'task_id': 'orchestration_20251203195311169717', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The persistent `Error during client shutdown: AssertionError()` error in the `execute_python_code` function suggests that the client-side shutdown logic (`kc.shutdown()`) might be attempting to operate on an already-closed or invalid connection. I will modify the `finally` block of the `execute_python_code` function to first attempt a graceful kernel shutdown via `kc.shutdown_request()`, and crucially, remove the explicit `kc.shutdown()` call to prevent the `AssertionError`. The kernel manager will then be responsible for terminating the kernel process via `km.shutdown_kernel(now=True)` for more robust cleanup.



In [48]:
import jupyter_client
import io
from contextlib import redirect_stdout, redirect_stderr
import time

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

def execute_python_code(code: str) -> dict:
    """
    Executes Python code in a sandboxed environment using a Jupyter kernel.
    Returns a dictionary containing stdout, stderr, and an error flag.
    """
    logger.info(f"Attempting to execute Python code:\n{code}")

    km = None
    kc = None
    try:
        # Use KernelManager to start a new kernel
        km = jupyter_client.KernelManager()
        km.start_kernel()

        # Connect a client to the kernel
        kc = km.client()
        kc.start_channels()
        kc.wait_for_ready()

        stdout_buffer = io.StringIO()
        stderr_buffer = io.StringIO()

        error_occurred = False

        try:
            # Execute the code interactively
            msg_id = kc.execute(code)

            # Poll for messages until idle or timeout
            while True:
                try:
                    msg = kc.get_iopub_msg(timeout=1)
                    msg_type = msg['msg_type']
                    content = msg['content']

                    if msg_type == 'stream':
                        if content['name'] == 'stdout':
                            stdout_buffer.write(content['text'])
                        elif content['name'] == 'stderr':
                            stderr_buffer.write(content['text'])
                    elif msg_type == 'error':
                        stderr_buffer.write(f"{content['ename']}: {content['evalue']}\n")
                        for tb_line in content.get('traceback', []):
                            stderr_buffer.write(f"  {tb_line}")
                        error_occurred = True
                    elif msg_type == 'status' and content['execution_state'] == 'idle':
                        break # Kernel is idle, execution is complete
                except jupyter_client.timeout.Empty: # No more messages in this loop iteration
                    break
                except Exception as e:
                    stderr_buffer.write(f"Error processing kernel message: {e}\n")
                    error_occurred = True
                    break

        except Exception as e:
            stderr_buffer.write(f"Execution error: {e}\n")
            error_occurred = True

        stdout_output = stdout_buffer.getvalue().strip()
        stderr_output = stderr_buffer.getvalue().strip()

        if error_occurred or stderr_output: # Consider stderr as an error if it contains content
            logger.error(f"Python code execution failed. Stderr:\n{stderr_output}")
        else:
            logger.info(f"Python code executed successfully. Stdout:\n{stdout_output}")

        return {
            "stdout": stdout_output,
            "stderr": stderr_output,
            "error": error_occurred or bool(stderr_output)
        }
    finally:
        # Shut down the client first, then the kernel manager
        if kc:
            try:
                # Attempt to send a shutdown request to the kernel
                kc.shutdown_request()
                # Wait a bit for the kernel to acknowledge or shut down
                # This can sometimes hang, so a short timeout is crucial.
                kc.wait_for_message(timeout=0.5) # Wait for shutdown reply (if any)
                logger.debug("Kernel shutdown request sent.")
            except Exception as e:
                logger.warning(f"Error sending kernel shutdown request via client: {e!r}")

            try:
                # Ensure client channels are stopped
                kc.stop_channels()
                logger.debug("Client channels stopped.")
            except Exception as e:
                logger.warning(f"Error stopping client channels: {e!r}")

            # Removed kc.shutdown() as it seems to be the source of AssertionError
            # The kernel manager shutdown should handle process termination.

        if km:
            try:
                # Ensure the kernel process is terminated by the manager
                if km.is_alive(): # Only try to shutdown if kernel is alive
                    km.shutdown_kernel(now=True) # Force immediate shutdown
                    logger.debug("Kernel manager shut down.")
            except Exception as e:
                logger.error(f"Error during kernel manager shutdown: {e!r}")

print("Python execution sandbox function 'execute_python_code' defined with improved shutdown handling.")

Python execution sandbox function 'execute_python_code' defined with improved shutdown handling.


**Reasoning**:
The previous demo workflows were executed with a previous version of the `execute_python_code` function and had a typo in agent instantiation. Now that `execute_python_code` has been further refined for robust kernel shutdown and previous typos in agent instantiation have been identified, I will re-run the complete demo workflows with the updated function and corrected agent assignments to verify full system functionality and accurate trace retrieval without errors.



In [49]:
import datetime

# Re-define OrchestratorAgent to return task_id (if not already updated from previous steps)
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# --- Demo Workflow Execution ---
print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components
# Re-instantiate MemoryManager, Tracer, and ContextManager to ensure fresh state
memory_manager = MemoryManager()
tracer = Tracer()
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (explicitly re-populate with latest definitions)
tools = {}
tools['google_search'] = google_search
tools['execute_python_code'] = execute_python_code # Ensure the latest version is used
tools['read_file'] = read_file
tools['write_file'] = write_file

print("Tools dictionary re-populated with latest function definitions.")

# 3. Instantiate agents
research_agent = ResearchAgent(memory_manager, tools, tracer)
search_agent = SearchAgent(memory_manager, tools, tracer) # Corrected instantiation
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents = {}
agents['research_agent'] = research_agent
agents['search_agent'] = search_agent # Corrected assignment
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated and assigned.")

# 4. Instantiate the OrchestratorAgent with the correct agents and tools
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")


OrchestratorAgent class updated to return result and task_id.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary re-populated with latest function definitions.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated and assigned.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Research Task (ID: o

Code-Execution Task Result:
{'stdout': 'Hello, NeuroChain Agents!\n3', 'stderr': '', 'error': False}

Trace for Code-Execution Task (ID: orchestration_20251203195402019730):
{'timestamp': '2025-12-03T19:54:02.019761', 'task_id': 'orchestration_20251203195402019730', 'event_type': 'start_trace', 'description': 'Orchestrating task: execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)', 'details': {}}
{'timestamp': '2025-12-03T19:54:02.019776', 'task_id': 'orchestration_20251203195402019730', 'event_type': 'orchestration_task_received', 'details': {'task': 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}
{'timestamp': '2025-12-03T19:54:02.019797', 'task_id': 'orchestration_20251203195402019730', 'event_type': 'delegation', 'details': {'agent': 'CodeRunnerAgent', 'code_snippet': 'print("Hello, NeuroChain Agents!")\nprint(1 + 2)'}}

Demo workflows completed.


**Reasoning**:
The `AttributeError` for `kc.shutdown_request()` indicates that this method is not available for `BlockingKernelClient`. I will modify the `execute_python_code` function to remove the problematic `kc.shutdown_request()` call and simplify the client shutdown to just `kc.stop_channels()`, relying on `km.shutdown_kernel(now=True)` for robust kernel termination. Then I will re-run the demo workflows to verify the fix.



In [50]:
import jupyter_client
import io
from contextlib import redirect_stdout, redirect_stderr
import time

# Ensure logger is available, it was initialized earlier
logger = logging.getLogger(__name__)

def execute_python_code(code: str) -> dict:
    """
    Executes Python code in a sandboxed environment using a Jupyter kernel.
    Returns a dictionary containing stdout, stderr, and an error flag.
    """
    logger.info(f"Attempting to execute Python code:\n{code}")

    km = None
    kc = None
    try:
        # Use KernelManager to start a new kernel
        km = jupyter_client.KernelManager()
        km.start_kernel()

        # Connect a client to the kernel
        kc = km.client()
        kc.start_channels()
        kc.wait_for_ready()

        stdout_buffer = io.StringIO()
        stderr_buffer = io.StringIO()

        error_occurred = False

        try:
            # Execute the code interactively
            msg_id = kc.execute(code)

            # Poll for messages until idle or timeout
            while True:
                try:
                    msg = kc.get_iopub_msg(timeout=1)
                    msg_type = msg['msg_type']
                    content = msg['content']

                    if msg_type == 'stream':
                        if content['name'] == 'stdout':
                            stdout_buffer.write(content['text'])
                        elif content['name'] == 'stderr':
                            stderr_buffer.write(content['text'])
                    elif msg_type == 'error':
                        stderr_buffer.write(f"{content['ename']}: {content['evalue']}\n")
                        for tb_line in content.get('traceback', []):
                            stderr_buffer.write(f"  {tb_line}")
                        error_occurred = True
                    elif msg_type == 'status' and content['execution_state'] == 'idle':
                        break # Kernel is idle, execution is complete
                except jupyter_client.timeout.Empty: # No more messages in this loop iteration
                    break
                except Exception as e:
                    stderr_buffer.write(f"Error processing kernel message: {e}\n")
                    error_occurred = True
                    break

        except Exception as e:
            stderr_buffer.write(f"Execution error: {e}\n")
            error_occurred = True

        stdout_output = stdout_buffer.getvalue().strip()
        stderr_output = stderr_buffer.getvalue().strip()

        if error_occurred or stderr_output: # Consider stderr as an error if it contains content
            logger.error(f"Python code execution failed. Stderr:\n{stderr_output}")
        else:
            logger.info(f"Python code executed successfully. Stdout:\n{stdout_output}")

        return {
            "stdout": stdout_output,
            "stderr": stderr_output,
            "error": error_occurred or bool(stderr_output)
        }
    finally:
        # Shut down the client channels first, then the kernel manager
        if kc:
            try:
                # Ensure client channels are stopped
                kc.stop_channels()
                logger.debug("Client channels stopped.")
            except Exception as e:
                logger.warning(f"Error stopping client channels: {e!r}")
            # Removed kc.shutdown() and kc.shutdown_request() as they cause issues.

        if km:
            try:
                # Ensure the kernel process is terminated by the manager
                if km.is_alive(): # Only try to shutdown if kernel is alive
                    km.shutdown_kernel(now=True) # Force immediate shutdown
                    logger.debug("Kernel manager shut down.")
            except Exception as e:
                logger.error(f"Error during kernel manager shutdown: {e!r}")

print("Python execution sandbox function 'execute_python_code' defined with improved shutdown handling.")

Python execution sandbox function 'execute_python_code' defined with improved shutdown handling.


**Reasoning**:
The previous steps included updates to the `execute_python_code` function and the `OrchestratorAgent` for more robust error handling and trace retrieval. Now I will re-run the full demo workflows, ensuring that all components are correctly initialized, the latest version of `execute_python_code` is used, and traces are accurately captured for verification.



In [51]:
import datetime

# Re-define OrchestratorAgent to return task_id (if not already updated from previous steps)
class OrchestratorAgent:
    def __init__(self, memory_manager, agents, tools, tracer):
        self.memory_manager = memory_manager
        self.agents = agents  # Dictionary of instantiated agents
        self.tools = tools    # Dictionary of available tools
        self.tracer = tracer # Store the tracer instance
        logger.info("OrchestratorAgent initialized.")

    def run(self, task: str) -> str:
        # Generate a unique task_id for each orchestration run
        task_id = f"orchestration_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}"
        self.tracer.start_trace(task_id=task_id, description=f"Orchestrating task: {task}")

        logger.info(f"Orchestration task initiated for: '{task}'")
        self.memory_manager.add_short_term_memory(f"Orchestration task: {task}")
        self.tracer.add_event(
            task_id=task_id,
            event_type="orchestration_task_received",
            details={
                "task": task
            }
        )

        task_lower = task.lower()

        result = ""
        if "research" in task_lower or "report on" in task_lower:
            query = task.replace("research", "").replace("report on", "").strip()
            logger.info(f"Delegating to ResearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "ResearchAgent",
                    "query": query
                }
            )
            result = self.agents['research_agent'].run(query)

        elif "search for" in task_lower or "find information on" in task_lower:
            query = task.replace("search for", "").replace("find information on", "").strip()
            logger.info(f"Delegating to SearchAgent for query: '{query}'")
            self.tracer.add_event(
                task_id=task_id,
                event_type="delegation",
                details={
                    "agent": "SearchAgent",
                    "query": query
                }
            )
            result = self.agents['search_agent'].run(query)

        elif "execute code" in task_lower or "run python" in task_lower:
            code_prefix_index = -1
            if "execute code:" in task_lower:
                code_prefix_index = task_lower.find("execute code:") + len("execute code:")
            elif "run python:" in task_lower:
                code_prefix_index = task_lower.find("run python:") + len("run python:")

            if code_prefix_index != -1:
                code_to_run = task[code_prefix_index:].strip()
                logger.info(f"Delegating to CodeRunnerAgent for code execution.\nCode:\n{code_to_run}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="delegation",
                    details={
                        "agent": "CodeRunnerAgent",
                        "code_snippet": code_to_run[:100] + "..." if len(code_to_run) > 100 else code_to_run
                    }
                )
                result = self.agents['code_runner_agent'].run(code_to_run)
            else:
                logger.warning(f"Code execution task specified, but no code found after 'execute code:' or 'run python:'. Task: {task}")
                self.tracer.add_event(
                    task_id=task_id,
                    event_type="error",
                    details={
                        "message": "Code execution task specified, but no code provided.",
                        "task": task
                    }
                )
                result = "Error: Code execution task specified, but no code provided."

        else:
            logger.info(f"No specific agent identified for task: '{task}'. Returning generic response.")
            self.tracer.add_event(
                task_id=task_id,
                event_type="no_specific_agent",
                details={
                    "task": task
                }
            )
            result = f"Orchestration completed for: {task}. No specialized agent invoked."

        # Return both the result and the task_id for trace retrieval
        return {"result": result, "task_id": task_id}

print("OrchestratorAgent class updated to return result and task_id.")

# --- Demo Workflow Execution ---
print("Initializing components and preparing demo workflows...")

# 1. Initialize necessary components
# Re-instantiate MemoryManager, Tracer, and ContextManager to ensure fresh state
memory_manager = MemoryManager()
tracer = Tracer()
context_manager = ContextManager(memory_manager)

print("MemoryManager, Tracer, and ContextManager instances are ready.")

# 2. Populate the tools dictionary (explicitly re-populate with latest definitions)
tools = {}
tools['google_search'] = google_search
tools['execute_python_code'] = execute_python_code # Ensure the latest version is used
tools['read_file'] = read_file
tools['write_file'] = write_file

print("Tools dictionary re-populated with latest function definitions.")

# 3. Instantiate agents
research_agent = ResearchAgent(memory_manager, tools, tracer)
search_agent = SearchAgent(memory_manager, tools, tracer) # Corrected instantiation
code_runner_agent = CodeRunnerAgent(memory_manager, tools, tracer)

agents = {}
agents['research_agent'] = research_agent
agents['search_agent'] = search_agent # Corrected assignment
agents['code_runner_agent'] = code_runner_agent

print("ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated and assigned.")

# 4. Instantiate the OrchestratorAgent with the correct agents and tools
orchestrator_agent = OrchestratorAgent(memory_manager, agents, tools, tracer)

print("OrchestratorAgent instantiated.")

print("\n--- Running Demo Workflows ---\n")

# 5. Define and execute a research demo task
research_task = 'research about the history of artificial intelligence'
print(f"Executing Research Task: '{research_task}'")
research_output = orchestrator_agent.run(research_task)
research_result = research_output['result']
research_task_id = research_output['task_id']

print(f"Research Task Result:\n{research_result}")

print(f"\nTrace for Research Task (ID: {research_task_id}):")
for event in tracer.get_trace(research_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 6. Define and execute a web search demo task
search_task = 'search for latest breakthroughs in quantum computing'
print(f"Executing Web Search Task: '{search_task}'")
search_output = orchestrator_agent.run(search_task)
search_result = search_output['result']
search_task_id = search_output['task_id']

print(f"Web Search Task Result:\n{search_result}")

print(f"\nTrace for Web Search Task (ID: {search_task_id}):")
for event in tracer.get_trace(search_task_id):
    print(event)

print("\n" + "="*50 + "\n")

# 7. Define and execute a code-execution demo task
code_task = 'execute code: print("Hello, NeuroChain Agents!")\nprint(1 + 2)'
print(f"Executing Code-Execution Task: '{code_task}'")
code_output = orchestrator_agent.run(code_task)
code_result = code_output['result']
code_task_id = code_output['task_id']

print(f"Code-Execution Task Result:\n{code_result}")

print(f"\nTrace for Code-Execution Task (ID: {code_task_id}):")
for event in tracer.get_trace(code_task_id):
    print(event)

print("\nDemo workflows completed.")


OrchestratorAgent class updated to return result and task_id.
Initializing components and preparing demo workflows...
MemoryManager class defined and initialized with short-term and long-term memory attributes.
ContextManager class defined and initialized.
MemoryManager, Tracer, and ContextManager instances are ready.
Tools dictionary re-populated with latest function definitions.
ResearchAgent, SearchAgent, and CodeRunnerAgent instantiated and assigned.
OrchestratorAgent instantiated.

--- Running Demo Workflows ---

Executing Research Task: 'research about the history of artificial intelligence'
Research Task Result:
Research Report on 'about the history of artificial intelligence':

Initial Query: about the history of artificial intelligence
Search Performed: Information about about the history of artificial intelligence
Key Findings (from simulated search): Simulated search results for: Information about about the history of artificial intelligence



Trace for Research Task (ID: o